In [1]:
library(igraph)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



In [22]:
G <- as.undirected(read.graph("galFiltered.gml", "gml"))

In [3]:
expressions <- read.table("galExpData.csv", sep = ",", header = T)

p_values <- expressions[,"gal1RGsig"]
names(p_values) <- expressions[,"GENE"]

z_score <- qnorm(1 - p_values)

genes <- V(G)$label[V(G)$label%in%names(z_score)]

z_score <- z_score[genes]

In [4]:
z_m <- z_score %*% t(z_score)
rownames(z_m) <- colnames(z_m)

In [5]:
exp_z <- exp(z_m)

In [6]:
exp_z <- exp_z / rowSums(exp_z)

In [23]:
A <- as_adjacency_matrix(G)

In [83]:
W <- 0.4 * diag(nrow(A)) + 0.5 * A %*% diag(1/degree(G)) + 0.1 * t(exp_z)

ERROR: Error: Matrices must have same dimensions in e1 + Matrix(e2)


In [40]:
matrix_multiply <- function(M, n) {
    if (n==0) return(diag(nrow=nrow(M)))
    return(M%*%matrix_multiply(M, n-1))
}

In [41]:
T <- matrix_multiply(W, 5)

In [51]:
rownames(T) <- V(G)$label
colnames(T) <- V(G)$label
T <- T[rownames(exp_z), colnames(exp_z)]

In [72]:
lambda <- 0.5

In [73]:
TZ <- (1 - lambda) * T + lambda *  t(exp_z)

In [75]:
TZ <- TZ / colSums(as.matrix(TZ))

In [79]:
TZ <- t(as.matrix(TZ))

In [81]:
write.csv(TZ, file="lambda=05.csv")